# Препроцессинг сцен Ландсат: маски облачности

В этом документе производится очистка сцен от облаков и их теней на базе канала BQA. [В блокноте](CMASK_exploration.ipynb) рассматривается другой подход к выделению облачности: облака ищутся самостоятельно на базе обучающей выборки.

In [ ]:
import os
import uuid

import utilites
reload(utilites)

from utilites import (
    get_grassdata_path,
    get_location_name,
    get_ll_location_name,
    get_location_path,
)


from grasslib import GRASS

grs = GRASS(gisbase='/usr/lib/grass70', 
            dbase=get_grassdata_path(), 
            location=get_location_name()
)

grs.grass.run_command('g.mapset', mapset='landsat', flags='c')
print grs.grass.read_command('g.mapset', flags='p')

Создадим функцию для маскирования. Она будет принимать на вход название растра BQA, список названий растров-каналов сцены, префикс результата и объект-обертку над GRASS, который будет производить обработку.

In [ ]:
def mask_scene(bqa_name, raster_list, prefix, grass):
    tempfile = uuid.uuid4().hex
    tempname = 'tmp' + uuid.uuid4().hex
    try:
        grass.run_command('i.landsat8.qc', 
                          cloud="Maybe,Yes", cloud_shadow='Maybe,Yes', 
                          output=tempfile)
        grass.run_command('r.reclass',
                           input=bqa_name, output=tempname, rules=tempfile)
        
        for rast_name in raster_list:
            out_name = prefix + rast_name
            expr = '%s=if(isnull(%s), null(), %s)' % (out_name, tempname, rast_name)
            grass.run_command('r.mapcalc',
                              expression=expr, overwrite=True)
    finally:
        os.unlink(tempfile)
        grass.run_command('g.remove', type='rast', name=tempname, flags='f')

Пример использования:

### Замаскируем все имеющиеся сцены:

In [ ]:
scenes = grs.grass.list_strings('rast', mapset='landsat', pattern='*_BQA')
# s is something like 'LC81130272014049LGN00_BQA@landsat'
scenes = [s[:-12] for s in scenes]
print scenes[:3]

In [ ]:
bands = ['_B1', '_B2', '_B3', '_B4', '_B5', '_B6', '_B7', '_B8', '_B9', '_B10', '_B11']
prefix = 'topo.toar_'
for s in scenes:
    print s
    map_list = [prefix + s + b for b in bands]
    grs.grass.run_command('g.region', rast=map_list[0])
    mask_scene(s + '_BQA', map_list, 'clean.', grs.grass)